# Notebook for running 1D-TDSE C code from Python

This notebook shows various analyses of the results of the multiscale model.


## Macroscopic part
* pulse propagation
* plasma channel
* XUV (Hankel)

## Microscopic insights
* Coherence maps
* One plane animation

## Load libraries and files

In [ ]:
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import os
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 


# %%capture
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

# %matplotlib agg
%matplotlib inline

In [ ]:
t = np.linspace(0,2*np.pi)
x = np.sin(t)

fig, ax = plt.subplots()
l, = ax.plot([0,2*np.pi],[-1,1])

animate = lambda i: l.set_data(t[:i], x[:i])

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))

from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
## import data
# h5file = os.path.join('/mnt','d','git','MMA-interactive','results.h5')
simulation = 'big'

h5file = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', simulation,'results.h5')

with h5py.File(h5file,'r') as f:


    # plot micro spectrum
    CUPRAD_res = dfC.get_data(f)

    zgrid_TDSE = f[MMA.paths['CTDSE_outputs'] +'/zgrid_coarse'][:]; Nz_TDSE = len(zgrid_TDSE)
    ogrid_TDSE = f[MMA.paths['CTDSE_outputs'] +'/omegagrid'][:]
    spectrum_middle =    f[MMA.paths['CTDSE_outputs'] +'/FSourceTerm'][Nz_TDSE//2,0,:,0] +\
                      1j*f[MMA.paths['CTDSE_outputs'] +'/FSourceTerm'][Nz_TDSE//2,0,:,1]

    
    
    # image = pp.figure_driver()    
    # image.sf = [pp.plotter() for k2 in range(16)]

    # image.sf[0].args = [ogrid_TDSE/mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'), np.abs(spectrum_middle)]
    # image.sf[0].method = plt.semilogy


    # pp.plot_preset(image)



    # ### plasma channel

    # CUPRAD_res.get_plasma(f)           

    # image = pp.figure_driver()    
    # image.sf = [pp.plotter() for k2 in range(16)]

    # symmetric_y, symmetric_data =  mn.symmetrize_y(CUPRAD_res.plasma.rgrid,CUPRAD_res.plasma.value_zrt[:,:,-1])

    #                 #  np.concatenate((np.flip(CUPRAD_res.plasma.value_zrt[:,1:,-1].T,axis=0),
    #                 #                   CUPRAD_res.plasma.value_zrt[:,:,-1].T                ),
    #                 #                  axis = 0)

    # # symmetric_y    = 1e6 * np.concatenate((np.flip(-CUPRAD_res.plasma.rgrid[1:]), CUPRAD_res.plasma.rgrid))

    # # image.sf[0].args = [1e3*CUPRAD_res.plasma.zgrid, 1e6*CUPRAD_res.plasma.rgrid, CUPRAD_res.plasma.value_zrt[:,:,-1].T]
    # image.sf[0].args = [1e3*CUPRAD_res.plasma.zgrid, 1e6*symmetric_y, symmetric_data.T]
    # image.sf[0].kwargs = {'shading' : 'auto'}
    # image.sf[0].colorbar.show = True
    # image.sf[0].method = plt.pcolormesh
    # image.ylim_args = [-150,150]


    # pp.plot_preset(image)
    
    
    # image2 = image

    # # local modulated density
    # local_density_modulation = CUPRAD_res.effective_neutral_particle_density *\
    #                              interpolate.interp1d(
    #                                 CUPRAD_res.density_mod_zgrid,
    #                                 CUPRAD_res.density_mod_profile_relative,
    #                                 bounds_error = False,
    #                                 fill_value = (CUPRAD_res.density_mod_profile_relative[0],
    #                                             CUPRAD_res.density_mod_profile_relative[-1]),
    #                                 copy = False
    #                                 )(CUPRAD_res.plasma.zgrid)

    # image2.sf[0].args[2] = 1e2*(mn.symmetrize_y(CUPRAD_res.plasma.rgrid,
    #                         CUPRAD_res.plasma.value_zrt[:,:,-1] /
    #                             np.outer(np.ones(len(CUPRAD_res.plasma.rgrid)),
    #                                      local_density_modulation).T
    #                         )[1]).T
    
    #                     #    1e2*CUPRAD_res.plasma.value_zrt[:,:,-1].T /\
    #                     #    np.outer(np.ones(len(CUPRAD_res.plasma.rgrid)),
    #                     #             local_density_modulation
    #                     #            )

    # pp.plot_preset(image2)
    

    print(matplotlib.rcParams['animation.embed_limit'])

    matplotlib.rcParams['animation.embed_limit'] = 200.

    print(matplotlib.rcParams['animation.embed_limit'])


    # fig, ax = plt.subplots()
    tlim = np.asarray((-25,25))
    rlim = 150
    k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res.tgrid,1.05*tlim)
    k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)

    fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [5, 1]})

    r_grid, sym_data = mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max],
                       (
                        HHG.ComputeCutoff(
                            mn.FieldToIntensitySI(CUPRAD_res.E_zrt[0,:k_r_max,k_t_min:k_t_max])/units.INTENSITYau,
                            mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
                            mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
                        )[1]
                       ).T)

    pc = ax1.pcolormesh(1e15*CUPRAD_res.tgrid[k_t_min:k_t_max], r_grid, sym_data.T, shading='auto')

    ax1.set_xlim((-25,25))
    ax1.set_ylim((-150,150))

    ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[0]) + ' mm')
    ax1.set_xlabel(r'$t~[\mathrm{fs}]$')
    ax1.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

    cbar = fig.colorbar(pc, ax=ax1)
    cbar.ax.set_ylabel(r'Intensity [harmonic cut-off]', rotation=90)


    ax2.plot(1e3*CUPRAD_res.density_mod_zgrid,
             CUPRAD_res.density_mod_profile_mbar)
    ax2.set_xlabel(r'$z~[\mathrm{mm}]$')
    ax2.set_ylabel(r'$p~[\mathrm{mbar}]$')

    progress_line, = ax2.plot([], [], 'r-')  

    ax2.fill_between(1e3*CUPRAD_res.density_mod_zgrid, CUPRAD_res.density_mod_profile_mbar,color='lightgrey')
    ax2.set_xlim((1e3*CUPRAD_res.density_mod_zgrid[0],1e3*CUPRAD_res.density_mod_zgrid[-1]))

    def update(frame):
        # Update the data
        data = (mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max], (
                HHG.ComputeCutoff(
                            mn.FieldToIntensitySI(CUPRAD_res.E_zrt[frame,:k_r_max,k_t_min:k_t_max])/units.INTENSITYau,
                            mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
                            mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
                        )[1]        
                ).T)[1]).T
               #(mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max], (CUPRAD_res.E_zrt[frame,:k_r_max,k_t_min:k_t_max]**2).T)[1]).T
               #np.abs(CUPRAD_res.E_zrt[frame,:,:])**2
        
        # Update the colors
        pc.set_array(data.ravel())
        pc.set_clim(data.min(), data.max())
        cbar.update_normal(pc)

        ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[frame]) + ' mm')

        # Update the progress indicator
        progress_line.set_data([1e3*CUPRAD_res.zgrid[frame], 1e3*CUPRAD_res.zgrid[frame]],
                                [CUPRAD_res.density_mod_profile_mbar.min(), CUPRAD_res.density_mod_profile_mbar.max()])

        return [pc, progress_line]


# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout()

ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(CUPRAD_res.zgrid), blit=True)

HTML(ani.to_jshtml())






In [ ]:
## import data
import matplotlib.colors as colors

h5file1 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', simulation,'results.h5')
h5file2 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', simulation,'results_Hankel.h5')

with h5py.File(h5file1,'r') as f1,  h5py.File(h5file2,'r') as f2:
    cummulative_field =    f2[MMA.paths['Hankel_outputs']+'/cummulative_field'][:,:,:,0] +\
                        1j*f2[MMA.paths['Hankel_outputs']+'/cummulative_field'][:,:,:,1]
    ogrid_Hankel = f2[MMA.paths['Hankel_outputs']+'/ogrid'][:]
    rgrid_Hankel = f2[MMA.paths['Hankel_outputs']+'/rgrid'][:]
    zgrid_Hankel = f2[MMA.paths['Hankel_outputs']+'/zgrid'][:]

    Hgrid_Hankel = ogrid_Hankel/CUPRAD_res.omega0


    image = pp.figure_driver()    
    image.sf = [pp.plotter() for k2 in range(16)]

    symmetric_y, symmetric_data =  mn.symmetrize_y(rgrid_Hankel,np.abs(cummulative_field[-1,:,:].T))

    # image.sf[0].args = [symmetric_data.T]
    image.sf[0].args = [ogrid_Hankel/CUPRAD_res.omega0,1e3*symmetric_y, symmetric_data.T]
    image.sf[0].kwargs = {'shading' : 'auto',
                          'norm'    : colors.LogNorm(vmin=symmetric_data.min(), vmax=symmetric_data.max())}
    image.sf[0].colorbar.show = True
    image.sf[0].method = plt.pcolormesh
    image.ylim_args = [-6,6]
    # image.xlim_args = [ogrid_Hankel[0]/CUPRAD_res.omega0,35]


    pp.plot_preset(image)

    # print(mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','lambdaSI')/(50.*rgrid_Hankel[1]))

    H_interest = np.asarray([19, 25, 37, 49]) 
    delta_H = 1.
    H_idx = [tuple(mn.FindInterval(Hgrid_Hankel,(H_interest[k1]-delta_H, H_interest[k1]+delta_H))) for k1 in range(len(H_interest))]

    max1 = np.max(np.abs(cummulative_field[:,:,H_idx[0][0]:H_idx[0][1]]),axis=(1,2))
    # max2 = np.max(np.abs(cummulative_field[:,:,H_idx[1][0]:H_idx[1][1]]),axis=(1,2))

    ### THIS SEEMS TO BE SLOW
    # H_max_interest = [np.max(np.abs(cummulative_field[:,:,H_idx[k1][0]:H_idx[k1][1]]),axis=(1,2)) for k1 in range(len(H_interest))]
    #                   np.max(np.abs(cummulative_field[:,H_idx[1][0]:H_idx[1][1],:]),axis=(1,2))]



    image = pp.figure_driver()    
    image.sf = [pp.plotter() for k2 in range(16)]

    for k1 in range(len(H_interest)):
        image.sf[k1].args = [zgrid_Hankel[1:],H_max_interest[k1][:]]

    pp.plot_preset(image)

    image = pp.figure_driver()    
    image.sf = [pp.plotter() for k2 in range(16)]

    image.sf[k1].args = [zgrid_Hankel[1:],H_max_interest[-1][:]]

    pp.plot_preset(image)
    

In [ ]:
# import matplotlib.pyplot as plt
# plt.rcParams["animation.html"] = "jshtml"
# import matplotlib.animation
# import numpy as np

# t = np.linspace(0,2*np.pi)
# x = np.sin(t)

# fig, ax = plt.subplots()
# l, = ax.plot([0,2*np.pi],[-1,1])

# animate = lambda i: l.set_data(t[:i], x[:i])

# ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))
# ani